# Clock Game Sync Word Finder

In [17]:
Numbers = [
    'dodici',
    'uno',
    'due',
    'tre',
    'quattro',
    'cinque',
    'sei',
    'sette',
    'otto',
    'nove',
    'dieci',
    'undici',
]

nNumbers = len(Numbers)

In [18]:
Automaton = [[len(n) % nNumbers, (len(n) + i) % nNumbers] for i, n in enumerate(Numbers)]
Automaton

[[6, 6],
 [3, 4],
 [3, 5],
 [3, 6],
 [7, 11],
 [6, 11],
 [3, 9],
 [5, 0],
 [4, 0],
 [4, 1],
 [5, 3],
 [6, 5]]

In [19]:
#Automaton for Italian Regions
Automaton = [
    [7, 11],
    [5, 3],
    [1, 3],
    [6, 0],
    [5, 1],
    [7, 6],
    [0, 7],
    [8, 11],
    [9, 5],
    [5, 4],
    [9, 8],
    [10, 8],
]

In [20]:
def Delta(state, move):
    return Automaton[state][move]

In [21]:
def Execute(word, starting_state):
    state = starting_state
    for c in word:
        state = Delta(state, int(c))
    return state

In [22]:
def hasKnownSyncWord(SyncWords, word):
    for w in SyncWords:
        if w in word:
            return True
    return False

In [23]:
maxLen = 15
iter = 2**maxLen
SyncWords = []

for n in range(iter):
    word = str(format(n + 2, 'b'))[1:]
    if not hasKnownSyncWord(SyncWords, word):
        outSet = set()

        for i in range(nNumbers):
            output = Execute(word, i)
            outSet.add(output)
            
        if(len(outSet) == 1):
            SyncWords.append(word)
            print(n, word.replace('0', 'A').replace('1', 'B') + ' ->', outSet)

9586 AABABABBBABAA -> {9}
9594 AABABABBBBBAA -> {9}
9691 AABABBBABBBAB -> {11}
9795 AABBAABAAABAB -> {11}
18499 AABAAAABAAABAB -> {11}
18562 AABAAABAAAABAA -> {7}
18570 AABAAABAAABBAA -> {7}
18627 AABAAABBAAABAB -> {11}
19011 AABABAABAAABAB -> {11}
19362 AABABBBABAABAA -> {7}
19547 AABBAAABABBBAB -> {11}
20338 AABBBBABBBABAA -> {9}
20346 AABBBBABBBBBAA -> {9}
